In [1]:
# 1.加载和预处理数据
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 加载鸢尾花数据集
iris = load_iris()
X, y = iris.data, iris.target

# 数据标准化和独热编码
# 标准化：将每个特征的值调整到均值为0，标准差为1
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 独热编码：将类别标签转化为独热向量 (如 0 -> [1, 0, 0])
encoder = OneHotEncoder()
y = encoder.fit_transform(y.reshape(-1, 1)).toarray()

# 划分训练集和测试集
# 将80%的数据用于训练，20%用于测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
# 2.定义激活函数和导数
# Sigmoid 激活函数
def sigmoid(x):
    """
    将输入值压缩到 (0, 1) 范围，用于激活神经元。
    """
    return 1 / (1 + np.exp(-x))

# Sigmoid 激活函数的导数
def sigmoid_derivative(x):
    """
    用于计算反向传播中的梯度。
    对 Sigmoid 函数求导的结果是 f'(x) = f(x) * (1 - f(x))。
    """
    return x * (1 - x)

In [3]:
# 3.定义 BP 神经网络类
class BPNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        """
        初始化神经网络参数，包括权重、偏置和学习率。
        参数：
        - input_size: 输入层节点数
        - hidden_size: 隐藏层节点数
        - output_size: 输出层节点数
        - learning_rate: 学习率，控制梯度更新步长
        """
        # 输入层到隐藏层的权重矩阵，初始化为随机值
        self.weights_input_hidden = np.random.uniform(-1, 1, (input_size, hidden_size))
        # 隐藏层的偏置
        self.bias_hidden = np.zeros((1, hidden_size))
        
        # 隐藏层到输出层的权重矩阵，初始化为随机值
        self.weights_hidden_output = np.random.uniform(-1, 1, (hidden_size, output_size))
        # 输出层的偏置
        self.bias_output = np.zeros((1, output_size))
        
        # 学习率
        self.learning_rate = learning_rate

    def forward(self, X):
        """
        执行前向传播，计算隐藏层和输出层的激活值。
        参数：
        - X: 输入数据
        返回：
        - final_output: 网络的最终输出
        """
        # 计算隐藏层的加权输入 (Z = X * W + b)
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        # 隐藏层的激活输出 (A = Sigmoid(Z))
        self.hidden_output = sigmoid(self.hidden_input)

        # 计算输出层的加权输入 (Z = H * W + b)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        # 输出层的激活输出 (A = Sigmoid(Z))
        self.final_output = sigmoid(self.final_input)

        return self.final_output

    def backward(self, X, y, output):
        """
        执行反向传播，计算误差并更新权重和偏置。
        参数：
        - X: 输入数据
        - y: 实际标签
        - output: 模型输出
        """
        # 输出层误差 (Error = y - output)
        output_error = y - output
        # 输出层梯度 (Delta = Error * f'(output))
        output_delta = output_error * sigmoid_derivative(output)

        # 隐藏层误差 (传播到隐藏层的误差)
        hidden_error = np.dot(output_delta, self.weights_hidden_output.T)
        # 隐藏层梯度 (Delta = Error * f'(hidden_output))
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_output)

        # 更新隐藏层到输出层的权重和偏置
        self.weights_hidden_output += self.learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.bias_output += self.learning_rate * np.sum(output_delta, axis=0, keepdims=True)

        # 更新输入层到隐藏层的权重和偏置
        self.weights_input_hidden += self.learning_rate * np.dot(X.T, hidden_delta)
        self.bias_hidden += self.learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)

    def train(self, X, y, epochs):
        """
        训练神经网络。
        参数：
        - X: 训练数据
        - y: 训练标签
        - epochs: 训练的迭代次数
        """
        for epoch in range(epochs):
            # 前向传播
            output = self.forward(X)

            # 反向传播
            self.backward(X, y, output)

            # 每隔100个 epoch 打印一次损失值
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))  # 均方误差
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        """
        使用训练好的网络进行预测。
        参数：
        - X: 测试数据
        返回：
        - 预测类别（最大概率对应的索引）
        """
        output = self.forward(X)
        return np.argmax(output, axis=1)  # 返回概率最大的类别索引


In [8]:
# 4.训练和评估模型
# 4.1 初始化神经网络
input_size = X_train.shape[1]  # 输入层节点数：特征数量
hidden_size = 2  # 隐藏层节点数：可以调节
output_size = y_train.shape[1]  # 输出层节点数：类别数量
learning_rate = 0.1  # 学习率

# 创建网络实例
bp_nn = BPNeuralNetwork(input_size, hidden_size, output_size, learning_rate)

# 训练神经网络
bp_nn.train(X_train, y_train, epochs=1000)

# 使用测试集进行预测
predictions = bp_nn.predict(X_test)

# 计算准确率
accuracy = np.mean(predictions == np.argmax(y_test, axis=1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 0, Loss: 0.22128556516365086
Epoch 100, Loss: 0.021410587809142765
Epoch 200, Loss: 0.013800814452345782
Epoch 300, Loss: 0.012100929959213873
Epoch 400, Loss: 0.011392964488187917
Epoch 500, Loss: 0.011017511806778143
Epoch 600, Loss: 0.010790199253609335
Epoch 700, Loss: 0.010640489751476679
Epoch 800, Loss: 0.010535921139959336
Epoch 900, Loss: 0.01045957534860003
Test Accuracy: 100.00%
